In [1]:
import requests
import pandas as pd
from datetime import datetime
import psycopg2
import time

In [2]:
conn = psycopg2.connect(database="postgres", user="postgres",
                        password="Aa@123456", host="127.0.0.1", port="5432")

In [9]:
res = {}
cur = conn.cursor()
cur.execute('''SELECT created_at, recommendation, price FROM Recommendations ORDER BY created_at DESC LIMIT 1''')
rows = cur.fetchall()
conn.commit()

if rows[0][1] == "BUY":
    res["price"] = rows[0][2]
else:
    res["price"] = 0.0
    
res["operation"] = rows[0][1]
res["datetime"] = str(rows[0][0]).split(" ")[1]

res

{'price': 34614.1, 'operation': 'BUY', 'datetime': '01:29:30'}

In [53]:
cur = conn.cursor()
cur.execute('''SELECT created_at, recommendation, price FROM Recommendations''')
rows = cur.fetchall()
conn.commit()

if rows[0][1] == "SELL":
    rows = rows[1:]

    
if rows[len(rows) -1][1] == "BUY":
    rows = rows[:-1]

In [54]:
df = pd.DataFrame(rows, columns=['time', 'recommendation', 'price'])
df["pct"] = 0
df["runing_amount"] = 0

In [55]:
df.head()

,time,recommendation,price,pct,runing_amount
0,2021-07-01 14:01:00,BUY,33245.25670,0,0
1,2021-07-01 14:13:30,SELL,33347.49895,0,0
2,2021-07-01 14:14:00,BUY,33396.16460,0,0
3,2021-07-01 14:14:30,SELL,33371.53160,0,0
4,2021-07-01 14:27:30,BUY,33435.43215,0,0


In [56]:
# Investment
amount= 100000
transactions_num = len(df)
transactions_fees = 0.5 # maybe we need to use if statements here
total_fees = transactions_num * transactions_fees

In [57]:
for i in range(len(df)):
    if df["recommendation"].iloc[i] == "SELL":
        df["pct"].iloc[i]=(df["price"].iloc[i]-df["price"].iloc[i-1])/df["price"].iloc[i-1]
        if i<=1:
            df["runing_amount"].iloc[i]=(1+df["pct"].iloc[i])*amount
        else:
            df["runing_amount"].iloc[i]=(1+df["pct"].iloc[i])*df["runing_amount"].iloc[i-2]

/Library/Python/3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [58]:
df

,time,recommendation,price,pct,runing_amount
0,2021-07-01 14:01:00,BUY,33245.256700,0.000000,0.000000
1,2021-07-01 14:13:30,SELL,33347.498950,0.003075,100307.539361
2,2021-07-01 14:14:00,BUY,33396.164600,0.000000,0.000000
3,2021-07-01 14:14:30,SELL,33371.531600,-0.000738,100233.552553
4,2021-07-01 14:27:30,BUY,33435.432150,0.000000,0.000000
...,...,...,...,...,...
59,2021-07-01 20:35:00,SELL,33477.054650,0.008371,100240.721559
60,2021-07-01 20:36:30,BUY,33414.041700,0.000000,0.000000
61,2021-07-01 20:37:30,SELL,33431.811650,0.000532,100294.030663
62,2021-07-01 20:39:00,BUY,33399.850425,0.000000,0.000000


In [51]:
rtn_pct=(df["runing_amount"].iloc[-1])/ amount * 100
#dollar profit/loss
profit_loss= df["runing_amount"].iloc[-1] - amount

In [52]:
print(rtn_pct,"% \n",profit_loss,"\n")

0.0 % 
 -100000.0 



In [ ]:
@btcData.route("/btc/profit/details", methods=["GET"])
def get_btc_profit_details():
    res = []
    cur = conn.cursor()
    cur.execute(
        '''SELECT created_at, recommendation, price FROM Recommendations ORDER BY created_at DESC LIMIT 6''')
    rows = cur.fetchall()
    conn.commit()
    
    rows.reverse()

    if len(rows) > 2:
        if rows[0][1] == "SELL":
            rows = rows[1:]

        if rows[len(rows) - 1][1] == "BUY":
            rows = rows[:-1]
            
    previous_buy = 0
    
    for i in range(len(rows)):
        current_res = {}

        if str(rows[i][1]) == "BUY":
            previous_buy = rows[i][2]
            current_res["profit/loss"] = 0.0
        else:
            current_res["profit/loss"] = rows[i][2] - previous_buy

        current_res["datetime"] = str(rows[i][0]).split(" ")[1]
        current_res["recommendation"] = str(rows[i][1])
        current_res["price"] = str(rows[i][2])
        res.append(current_res)
        
    result = jsonify(res)
    result.headers.add('Access-Control-Allow-Origin', '*')

    return result

In [ ]:
@btcData.route("/btc/profit/details/lastOperation", methods=["GET"])
def get_btc_profit_details_last_operation():
    res = {}
    cur = conn.cursor()
    cur.execute(
        '''SELECT created_at, recommendation, price FROM Recommendations ORDER BY created_at DESC LIMIT 1''')
    rows = cur.fetchall()
    conn.commit()

    if rows[0][1] == "BUY":
        res["price"] = rows[0][2]
    else:
        res["price"] = 0.0
        
    res["operation"] = rows[0][1]
    res["datetime"] = str(rows[0][0]).split(" ")[1]

    result = jsonify(res)
    result.headers.add('Access-Control-Allow-Origin', '*')

    return result